In [24]:
# Import libraries

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd
import numpy as np
import joblib

In [27]:
import pandas as pd
df = pd.read_csv(r"C:\CodigosGithub\Hospital_LOS_Prediction\data\hospital_cleaned.csv")
df.columns


Index(['facility_name', 'age_group', 'gender', 'race', 'ethnicity',
       'length_of_stay', 'type_of_admission', 'patient_disposition',
       'discharge_year', 'ccsr_diagnosis_code', 'ccsr_diagnosis_description',
       'apr_drg_code', 'apr_drg_description', 'apr_mdc_code',
       'apr_mdc_description', 'apr_severity_of_illness_code',
       'apr_severity_of_illness_description', 'apr_risk_of_mortality',
       'apr_medical_surgical_description', 'payment_typology_1',
       'emergency_department_indicator', 'total_charges', 'total_costs'],
      dtype='object')

## Target variable

**length_of_stay**

## Predictor candidates

**Categorical:**

- age_group
- gender
- race
- ethnicity
- type_of_admission
- patient_disposition
- ccsr_diagnosis_description
- apr_severity_of_illness_description
- apr_risk_of_mortality
- apr_medical_surgical_description
- payment_typology_1
- emergency_department_indicator

In [28]:
# Sample 100,000 records for tuning and testing
df_sample = df.sample(n=min(100_000, len(df)), random_state=42)

# Define features and target variable
target = 'length_of_stay'
categorical_features = [
    'age_group', 'gender', 'race', 'ethnicity',
    'type_of_admission', 'patient_disposition',
    'ccsr_diagnosis_description', 'apr_severity_of_illness_description',
    'apr_risk_of_mortality', 'apr_medical_surgical_description',
    'payment_typology_1', 'emergency_department_indicator'
]

# Split data into features (X) and target (y)
X = df_sample[categorical_features]
y = df_sample[target]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
# Preprocessing pipeline
preprocessor = ColumnTransformer(transformers=[
    ('categorical', OneHotEncoder(handle_unknown='ignore'), categorical_features),
])

# Define the XGBoost Regressor (GPU-enabled)
xgb_model = XGBRegressor(
    tree_method='hist',      # modern XGBoost method
    device='cuda',           # use CUDA GPU
    n_estimators=100,
    random_state=42
)

# Combine preprocessing and model in a pipeline
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', xgb_model)
])

# Fit the pipeline
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = pipeline.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Model evaluation:")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f} days")
print(f"Mean Absolute Error (MAE): {mae:.2f} days")
print(f"R² Score: {r2:.4f}")

Model evaluation:
Root Mean Squared Error (RMSE): 6.91 days
Mean Absolute Error (MAE): 3.26 days
R² Score: 0.3276


These results demonstrate the advantage of using machine learning to predict hospital stays based on different variables. Now that we know this, let's find the model that generates the best results and optimize the hyperparameters.

In [32]:
# Define models to compare
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100, random_state=42),
    "XGBoost (GPU)": XGBRegressor(tree_method='hist', device='cuda', n_estimators=100, random_state=42)
}

results = []

# Loop through models and evaluate
for name, model in models.items():
    pipeline = Pipeline(steps=[
        ('preprocessing', preprocessor),
        ('model', model)
    ])
    
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        'Model': name,
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2
    })

# Display results
results_df = pd.DataFrame(results).sort_values(by='RMSE')
results_df.reset_index(drop=True, inplace=True)
results_df

,Model,RMSE,MAE,R²
0,XGBoost (GPU),6.909996,3.261811,0.327648
1,Gradient Boosting,7.035038,3.392848,0.303095
2,Linear Regression,7.180826,3.583337,0.273911
3,Random Forest,7.377029,3.431734,0.233691


**Random Forest** and **XGBoost** showed the best results. Before training the model with all the data, we will do **hyperparameter tuning** with the sample of 100,000

# Hiperparameter Tuning

In [33]:

# Common preprocessor
preprocessor = ColumnTransformer([
    ('categorical', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

# -----------------------------
# Random Forest Tuning
# -----------------------------
rf_model = RandomForestRegressor(random_state=42, n_jobs=-1)
rf_pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('model', rf_model)
])

rf_params = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [10, 20, 30],
    'model__min_samples_split': [2, 5],
    'model__min_samples_leaf': [1, 2]
}

rf_search = RandomizedSearchCV(
    rf_pipeline,
    rf_params,
    n_iter=10,
    cv=3,
    scoring='neg_root_mean_squared_error',
    verbose=1,
    n_jobs=-1
)

rf_search.fit(X_train, y_train)
rf_best_model = rf_search.best_estimator_

# -----------------------------
# XGBoost Tuning (GPU)
# -----------------------------
xgb_model = XGBRegressor(tree_method='hist', device='cuda', random_state=42)
xgb_pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('model', xgb_model)
])

xgb_params = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [3, 6, 10],
    'model__learning_rate': [0.01, 0.1, 0.3],
    'model__subsample': [0.8, 1.0]
}

xgb_search = RandomizedSearchCV(
    xgb_pipeline,
    xgb_params,
    n_iter=10,
    cv=3,
    scoring='neg_root_mean_squared_error',
    verbose=1,
    n_jobs=-1
)

xgb_search.fit(X_train, y_train)
xgb_best_model = xgb_search.best_estimator_

# -----------------------------
# Evaluate and Save Models
# -----------------------------
def evaluate_model(model, X_test, y_test, name):
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"{name} Evaluation:")
    print(f"RMSE: {rmse:.4f} days")
    print(f"MAE : {mae:.4f} days")
    print(f"R²  : {r2:.4f}")
    return rmse, mae, r2

print("\n--- Final Evaluation on Sampled Test Set ---\n")
evaluate_model(rf_best_model, X_test, y_test, "Random Forest")
evaluate_model(xgb_best_model, X_test, y_test, "XGBoost")

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits

--- Final Evaluation on Sampled Test Set ---

Random Forest Evaluation:
RMSE: 6.9861 days
MAE : 3.3720 days
R²  : 0.3127
XGBoost Evaluation:
RMSE: 6.9216 days
MAE : 3.2934 days
R²  : 0.3254


(np.float64(6.92158235572827), 3.2934014797210693, 0.3253914713859558)

In [35]:
print("Best hyperparameters for XGBoost:")
print(xgb_search.best_params_)

Best hyperparameters for XGBoost:
{'model__subsample': 1.0, 'model__n_estimators': 100, 'model__max_depth': 6, 'model__learning_rate': 0.1}


The best model seems to be **XGBoost** with the following hyperparameters:

- model_subsample = 1
- model_n_estimators = 100
- model_max_depth = 6
- model_learning_rate = 0.1

Now, I'm going to train this model with all the data.

# Final Model

In [37]:
# Split data into features (X) and target (y)
X = df[categorical_features]
y = df[target]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing pipeline
preprocessor = ColumnTransformer([
    ('categorical', OneHotEncoder(handle_unknown='ignore'), categorical_features),
])

# Best hyperparameters from tuning
best_xgb = XGBRegressor(
    tree_method='hist',
    device='cuda',
    n_estimators=100,          
    max_depth=6,              
    learning_rate=0.1,         
    subsample=1.0,             
    random_state=42
)

# Full pipeline
final_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', best_xgb)
])

# Train on full dataset
print("🚀 Training XGBoost model on full dataset...")
final_pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = final_pipeline.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Model evaluation:")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f} days")
print(f"Mean Absolute Error (MAE): {mae:.2f} days")
print(f"R² Score: {r2:.4f}")

🚀 Training XGBoost model on full dataset...
Model evaluation:
Root Mean Squared Error (RMSE): 6.71 days
Mean Absolute Error (MAE): 3.21 days
R² Score: 0.3710


### Model Evaluation Summary

The final **XGBoost** model was trained using the entire dataset with the best hyperparameters identified during tuning.

**Performance on the validation set:**

- Root Mean Squared Error (RMSE): **6.71 days**
- Mean Absolute Error (MAE): **3.21 days**
- R² Score: **0.3710**

These results indicate that the model can explain approximately **37% of the variability** in patient length of stay (LOS), with an average error of **just over 3 days**.

### Conclusion

Although the performance is acceptable for a first version, it will likely improve by:

- Engineering new relevant features (e.g., diagnosis grouping, seasonal trends, bed availability)
- Including external variables (e.g., hospital type, prior visits, socioeconomic indicators)
- Trying advanced models (e.g., temporal models or ensemble stacking)

For now, this version will be used to power the **bed assignment optimization model**, but continuous refinement is expected in future iterations.


In [38]:
# Save the pipeline (preprocessor + trained XGBoost)
joblib.dump(final_pipeline, r"C:\CodigosGithub\Hospital_LOS_Prediction\models\xgb_final_pipeline.pkl")


['C:\\CodigosGithub\\Hospital_LOS_Prediction\\models\\xgb_final_pipeline.pkl']